

# **Origen de los datos**

Los datos obtenidos para este proyecto corresponden a un conjunto de 500 imágenes de documentos de identidad. De estas, 250 imágenes representan documentos reales y las otras 250 son imágenes generadas artificialmente, donde se superponen datos falsos sobre documentos auténticos. Estos datos fueron recopilados de un servicio de almacenamiento en la nube llamado AWS.

A continuación, se presenta un ejemplo de cómo se descargaron los datos desde Amazon S3 hacia un entorno local (por sencibilidad de datos y credenciales no se adjuntan solo se mencionan):


```
import boto3

# Establecer las credenciales de AWS
access_key = 'TU_ACCESS_KEY'
secret_key = 'TU_SECRET_KEY'
region_name = 'TU_REGION_NAME'

# Crear una instancia del cliente de AWS S3
s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)

# Nombre del bucket de S3 donde están almacenados los datos
bucket_name = 'NOMBRE_DEL_BUCKET'

# Directorio local donde se guardarán los datos descargados
local_directory = 'DIRECTORIO_LOCAL'

# Descargar los archivos desde S3 al directorio local
response = s3_client.download_file(bucket_name, 'CARPETA_EN_S3/nombre_del_archivo.jpg', f'{local_directory}/nombre_del_archivo.jpg')


```



Una vez que los datos se descargaron al entorno local, se organizaron en una carpeta compartida en Google Drive para facilitar el acceso y la colaboración entre los miembros del proyecto. Esto se hizo para que todos los involucrados pudieran acceder fácilmente a los datos y trabajar en conjunto en el desarrollo del proyecto.

En resumen, se recopilaron 500 imágenes de documentos de identidad, de las cuales 250 son documentos reales y 250 son imágenes generadas artificialmente. Estos datos fueron obtenidos de AWS mediante una descarga desde Amazon S3 hacia un entorno local. Posteriormente, se organizaron en una carpeta compartida en Google Drive para facilitar el acceso y la colaboración en el proyecto.

# Conexion a google drive

In [15]:
!apt install tree


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 46 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (125 kB/s)
Selecting previously unselected package tree.
(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack .../tree_1.8.0-1_amd64.deb ...
Unpacking tree (1.8.0-1) ...
Setting up tree (1.8.0-1) ...
Processing triggers for man-db (2.9.1-1) ...


In [6]:
#descarga del corpus
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_data  = '/content/drive/MyDrive/DIPLOMADO/DATA-SUPER'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/MyDrive/DIPLOMADO/DATA-SUPER
!tree -L 1

/content/drive/MyDrive/DIPLOMADO/DATA-SUPER
.
├── FALSE
└── TRUE

2 directories, 0 files


In [19]:
import os
import random
from PIL import Image, ImageDraw, ImageFilter
import xml.etree.ElementTree as ET
import os
def procesar_imagenes(ruta):
    # Obtener la lista de archivos en la ruta
    archivos = os.listdir(ruta)

    # Filtrar solo los archivos de imagen (por extensión)
    archivos_imagen = [archivo for archivo in archivos if archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    # Seleccionar aleatoriamente 5 imágenes
    imagenes_seleccionadas = random.sample(archivos_imagen, 5)

    for imagen in imagenes_seleccionadas:
        # Leer la imagen
        ruta_imagen = os.path.join(ruta, imagen)
        img = Image.open(ruta_imagen)

        # Leer el archivo XML asociado
        ruta_xml = os.path.splitext(ruta_imagen)[0] + '.xml'
        if os.path.isfile(ruta_xml):
            # Parsear el archivo XML
            tree = ET.parse(ruta_xml)
            root = tree.getroot()

            # Obtener las coordenadas del recuadro
            coordenadas = []
            for obj in root.iter('object'):
                xmin = int(obj.find('xmin').text)
                ymin = int(obj.find('ymin').text)
                xmax = int(obj.find('xmax').text)
                ymax = int(obj.find('ymax').text)
                coordenadas.append((xmin, ymin, xmax, ymax))

            # Dibujar los recuadros en la imagen
            draw = ImageDraw.Draw(img)
            for coord in coordenadas:
                draw.rectangle(coord, outline='red')

            # Aplicar un filtro de difuminado a las áreas fuera de los recuadros
            blurred_img = img.filter(ImageFilter.BoxBlur(10))

            # Mostrar la imagen original con los recuadros y la imagen difuminada
            img.show()
            blurred_img.show()
        else:
            print(f"No se encontró el archivo XML asociado a la imagen: {imagen}")


In [23]:
print(os.path.join(path_data,'TRUE'))
procesar_imagenes(os.path.join(path_data,'TRUE'))


/content/drive/MyDrive/DIPLOMADO/DATA-SUPER/TRUE
No se encontró el archivo XML asociado a la imagen: 12166173K.jpg
No se encontró el archivo XML asociado a la imagen: 74061955.jpg
No se encontró el archivo XML asociado a la imagen: 13898365K.jpg
No se encontró el archivo XML asociado a la imagen: front (46).png
No se encontró el archivo XML asociado a la imagen: 43057305F.JPG


In [ ]:
procesar_imagenes(os.path.join(path_data,'FALSE'))

In [ ]:
import os
import cv2
import argparse
from lxml import etree
import xml.etree.ElementTree as ET
from tqdm import tqdm
import subprocess

def generateDict(input_path):
    dict_imgs = dict()
    for file_i in os.listdir(input_path):
        if '.ini' in file_i:
            continue
        if 'DS_Store' in file_i:
            continue
        if '.xml' not in file_i:
            img_lst_i = file_i.split('.')
            if len(img_lst_i) == 2:
                name_img_i,formato_img_i = img_lst_i[0], img_lst_i[1]
            else:
                name_img_i, formato_img_i = ".".join(img_lst_i[:len(img_lst_i)-1]), img_lst_i[len(img_lst_i)-1]
            dict_imgs[name_img_i]=formato_img_i
    return dict_imgs

def get_region_coordinates(xml_path):
    """
    Función que lee un archivo XML y devuelve las coordenadas de la región de interés.
    """
    # Parseamos el archivo xml para obtener las coordenadas de recorte
    tree = ET.parse(xml_path)
    root = tree.getroot()
    xmin = int(root.find("./object/bndbox/xmin").text)
    ymin = int(root.find("./object/bndbox/ymin").text)
    xmax = int(root.find("./object/bndbox/xmax").text)
    ymax = int(root.find("./object/bndbox/ymax").text)
    print(xmin, ymin, xmax, ymax)
    return xmin, ymin, xmax, ymax

def main_cut(dir):
    directorio_actual = os.path.dirname(os.path.abspath(__file__))
    print(directorio_actual)

    # Crea la carpeta cut si no existe
    os.makedirs(os.path.join(dir,"cut"), exist_ok=True)
    input_path = os.path.join(os.getcwd(), dir)
    # os.makedirs(os.path.join(dir,"cut"), exist_ok=True)
    dict_imgs=generateDict(input_path)

    for name_i, formato_img_i in tqdm(dict_imgs.items(), total=len(dict_imgs)):
        if name_i != '':
    # for xml_file in xml_files:
        # Obtiene la ruta de la imagen correspondiente
            image_path = os.path.join(dir, name_i+'.'+formato_img_i)
            print(image_path)

            # Lee la imagen con OpenCV
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: could not load image {image_path}")
                continue

            # Obtiene las coordenadas de la región de interés
            print(os.path.join(dir, name_i))
            if not os.path.isfile(os.path.join(dir, name_i +".xml")):
                continue
            xmin, ymin, xmax, ymax = get_region_coordinates(os.path.join(dir, name_i +".xml"))
            print(xmin, ymin, xmax, ymax)
            # Recorta la imagen y guarda la imagen recortada en la carpeta cut
            try:
                if xmax-10 < image.shape[0] and   ymax-10 < image.shape[1]:
                    cut_image = image[ymin-10:ymax+10, xmin-15:xmax+15]
                else:
                    cut_image = image[ymin:ymax, xmin:xmax]
                cv2.imwrite(os.path.join(dir, "cut", name_i + "."+ formato_img_i), cut_image)
            except:
                cut_image = image[ymin:ymax, xmin:xmax]
                cv2.imwrite(os.path.join(dir, "cut", name_i + "."+ formato_img_i), cut_image)




In [ ]:
main_cut()

# **Funcion para generar de data set train, test y validation**

Este generador de datos tomará las imágenes de las carpetas "TRUE" y "FALSE", las escalará a un tamaño de 250x250 píxeles, las normalizará y realizará aumentación de datos. Luego, generará conjuntos de entrenamiento, prueba y validación con las proporciones mencionadas, mezclando la información de las dos carpetas.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_generator(data_dir, target_size=(250, 250), batch_size=32):
    # Crear generador de aumentación de datos
    datagen = ImageDataGenerator(
        rescale=1.0 / 255.0,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Obtener las rutas de las imágenes de las carpetas "TRUE" y "FALSE"
    true_dir = os.path.join(data_dir, "TRUE")
    false_dir = os.path.join(data_dir, "FALSE")

    # Obtener las listas de archivos de las carpetas
    true_files = [os.path.join(true_dir, file) for file in os.listdir(true_dir)]
    false_files = [os.path.join(false_dir, file) for file in os.listdir(false_dir)]

    # Mezclar las listas de archivos
    all_files = true_files + false_files
    random.shuffle(all_files)

    # Calcular el número de muestras en cada conjunto (train, test, validación)
    num_samples = len(all_files)
    num_train = int(0.6 * num_samples)
    num_test = int(0.2 * num_samples)
    num_val = num_samples - num_train - num_test

    # Dividir las rutas de los archivos en conjuntos de train, test y validación
    train_files = all_files[:num_train]
    test_files = all_files[num_train:num_train+num_test]
    val_files = all_files[num_train+num_test:]

    # Crear generadores de datos para cada conjunto
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    test_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=True
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=True
    )

    return train_generator, test_generator, val_generator

# Ruta que contiene las carpetas "TRUE" y "FALSE" con las imágenes
data_dir = "ruta_a_tu_carpeta_principal"

# Crear el generador de datos
train_generator, test_generator, val_generator = create_data_generator(data_dir)
